In [4]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1382423


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [16]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
def encode(s): return [stoi[c] for c in s]
def decode(l): return ''.join([itos[i] for i in l])


print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

In [22]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [23]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when the input is {context} the target: {target}")

when the input is tensor([18]) the target: 47
when the input is tensor([18, 47]) the target: 56
when the input is tensor([18, 47, 56]) the target: 57
when the input is tensor([18, 47, 56, 57]) the target: 58
when the input is tensor([18, 47, 56, 57, 58]) the target: 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [36]:
torch.manual_seed(42)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs: ")
print(xb.shape)
print(xb)

print("target: ")
print(yb.shape)
print(yb)

print('--------------------')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t+1]
        target = yb[b, t]
        print(f"when the input is {context.tolist()} the target: {target}")

inputs: 
torch.Size([4, 8])
tensor([[54, 43, 39, 49,  1, 39, 45, 39],
        [43, 56, 57, 11,  1, 61, 46, 53],
        [43,  1, 46, 39, 58, 46,  1, 58],
        [43,  1, 47, 57,  1, 57, 53,  1]])
target: 
torch.Size([4, 8])
tensor([[43, 39, 49,  1, 39, 45, 39, 47],
        [56, 57, 11,  1, 61, 46, 53,  6],
        [ 1, 46, 39, 58, 46,  1, 58, 56],
        [ 1, 47, 57,  1, 57, 53,  1, 50]])
--------------------
when the input is [54] the target: 43
when the input is [54, 43] the target: 39
when the input is [54, 43, 39] the target: 49
when the input is [54, 43, 39, 49] the target: 1
when the input is [54, 43, 39, 49, 1] the target: 39
when the input is [54, 43, 39, 49, 1, 39] the target: 45
when the input is [54, 43, 39, 49, 1, 39, 45] the target: 39
when the input is [54, 43, 39, 49, 1, 39, 45, 39] the target: 47
when the input is [43] the target: 56
when the input is [43, 56] the target: 57
when the input is [43, 56, 57] the target: 11
when the input is [43, 56, 57, 11] the target: 1

In [37]:
print(xb)

tensor([[54, 43, 39, 49,  1, 39, 45, 39],
        [43, 56, 57, 11,  1, 61, 46, 53],
        [43,  1, 46, 39, 58, 46,  1, 58],
        [43,  1, 47, 57,  1, 57, 53,  1]])


In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(137)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B,T,C) Batch Time Channel
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6978, grad_fn=<NllLossBackward0>)

N$;UE3Fts
On,EXd;QMBxdjO:EvGyI!GLOEx&MBcLuTZK.lkq,-Gu,pwxdu-VqIy&Hqp$e?liHHKJPXBbYgbQJgIaCd$GtFcZ'mA
